# Philip Carr
# CS/CNS/EE_156a_Homework_4_Code_Part_2 (Jupyter Notebook)

Code for Generalizaition Error (Problems 4-7)

In [1]:
import random as rn
import numpy as np
from matplotlib import pyplot as plt

In [2]:
def get_random_x_values(N):
    '''
    Return a list of randomly generated x values within
    the region [-1, 1].
    
    Return type: list of x values (each point is a list).
    '''
    random_x_values = []
    for n in range(N):
        random_x_values.append(rn.uniform(-1, 1))
    return np.array(random_x_values)

In [3]:
def mean_squared_error(estimated_values, true_values):
    """
    Return the mean squared error between arrays of estimated and
    true values.
    
    Return type: float.
    """
    return np.sqrt(np.mean((estimated_values - true_values)
                   * (estimated_values - true_values)))

In [4]:
def find_best_a_value_v1(x_values, a_lower_bound=-5, a_upper_bound=5,
                         n_a_values=1000):
    """
    Return the value of a in g(x) = ax that minimizes the mean
    squared error between the hypothesis function g(x) and the target
    function (f(x) = sin(pi * x)).
    
    Return type: float.
    """
    a_domain = np.linspace(a_lower_bound, a_upper_bound,
                           num=n_a_values)
    mean_squared_errors = np.zeros(n_a_values, dtype=float)
    for i in range(len(a_domain)):
        estimated_values = a_domain[i] * x_values
        true_values = np.sin(np.pi * x_values)
        mean_squared_errors[i] = mean_squared_error(estimated_values,
                                                    true_values)
    min_mean_squared_error_index = \
        np.where(mean_squared_errors == min(mean_squared_errors))[0]
    return a_domain[min_mean_squared_error_index]

In [5]:
def bias_and_variance_analysis_v1(trials=10000):
    """
    Print the value of a_hat in g_bar(x) = a_hat * x that minimizes
    the mean squared error between the hypothesis function g(x) and
    the target function (f(x) = sin(pi * x)), and print the bias
    and variance associated with a_hat and g_bar(x).
    
    Return type: None.
    """
    a_values = np.zeros(trials, dtype=float)
    for i in range(trials):
        x_values = get_random_x_values(2)
        a_values[i] = find_best_a_value_v1(x_values)
        
    a_average = np.mean(a_values)
    print("a_hat =", a_average)
    
    new_x_values = get_random_x_values(trials)
    g_bar_x_values = a_average * new_x_values
    
    f_x_values = np.sin(np.pi * new_x_values)
    bias = np.mean((g_bar_x_values - f_x_values)
                   * (g_bar_x_values - f_x_values))
    print("bias =", bias)
    
    average_g_D_minus_g_bar_values = np.zeros(trials, dtype=float)
    for i in range(trials):
        g_D_x_values = a_values[i] * new_x_values
        average_g_D_minus_g_bar_value = \
            np.mean((g_D_x_values - g_bar_x_values)
                    * (g_D_x_values - g_bar_x_values))
        average_g_D_minus_g_bar_values[i] = \
            average_g_D_minus_g_bar_value
    variance = np.mean(average_g_D_minus_g_bar_values)
    print("variance =", variance)

For Problems 4-6

In [6]:
bias_and_variance_analysis_v1()

a_hat = 1.4304334334334337
bias = 0.27222391670753693
variance = 0.23808123873991735


In [7]:
def find_best_a_value_v2(x_values, g_function, a_lower_bound=-5,
                         a_upper_bound=5, n_a_values=100,
                         b_lower_bound=-1, b_upper_bound=2,
                         n_b_values=10):
    """
    Return the value of a and b in for a given g(x) that minimizes
    the mean squared error between the hypothesis function g(x)
    and the target function (f(x) = sin(pi * x)).
    
    Return type: tuple of floats.
    """
    a_b_domain = []#np.linspace(a_lower_bound, a_upper_bound,
                    #       num=n_a_values)
    delta_a = (a_lower_bound - a_upper_bound) / n_a_values
    delta_b = (b_lower_bound - b_upper_bound) / n_b_values
    mean_squared_errors = []#np.zeros(n_a_values, dtype=float)
    for n in range(n_a_values):
        a_n = float(a_lower_bound) + float(n) * delta_a
        row_a_b = []
        row_error = []
        for m in range(n_b_values):
            b_m = float(b_lower_bound) + float(m) * delta_b
            row_a_b.append((a_n, b_m))
            if g_function == 'b':
                estimated_values = a_b_domain[n][m][1]
            elif g_function == 'ax':
                estimated_values = a_b_domain[n][m][0] * x_values
            elif g_function == 'ax + b':
                estimated_values = a_b_domain[n][m][0] * x_values \
                                   + a_b_domain[n][m][1]
            elif g_function == 'ax^2':
                estimated_values = a_b_domain[n][m][0] * x_values \
                                   * x_values
            elif g_function == 'ax^2 + b':
                estimated_values = a_b_domain[n][m][0] * x_values \
                                   * x_values + a_b_domain[n][m][1]
            else:
                raise ValueError("Invalid g_function: " + g_function)
            true_values = np.sin(np.pi * x_values)
            row_error.append(mean_squared_error(estimated_values,
                                                true_values))
        a_b_domain.append(row_a_b)
        mean_squared_errors.append(row_error)
        
    min_n = -1
    min_m = -1
    min_error = 99999
    for n in range(n_a_values):
        for m in range(n_b_values):
            if min_error > mean_squared_errors[n][m]:
                min_error = mean_squared_errors[n][m]
                min_n = n
                min_m = m
    
    return a_b_domain[min_n][min_m]

In [8]:
def bias_and_variance_analysis_v2(g_function, trials=10000):
    """
    Print the value of a_hat and b_hat in g_bar(x) that minimizes
    the mean squared error between the hypothesis function g(x) and
    the target function (f(x) = sin(pi * x)), and print the bias
    and variance associated with a_hat, b_hat, and g_bar(x).
    
    Return type: float.
    """
    a_values = np.zeros(trials, dtype=float)
    b_values = np.zeros(trials, dtype=float)
    for i in range(trials):
        x_values = get_random_x_values(2)
        a_values[i], b_values[i] = \
            find_best_a_value_v2(x_values, g_function)
        
    a_average = np.mean(a_values)
    print("a_hat =", a_average)
    
    b_average = np.mean(b_values)
    print("b_hat =", b_average)
    
    new_x_values = get_random_x_values(trials)
    if g_function == 'b':
        g_bar_x_values = b_average
    elif g_function == 'ax':
        g_bar_x_values = a_average * new_x_values
    elif g_function == 'ax + b':
        g_bar_x_values = a_average * new_x_values \
                           + b_average
    elif g_function == 'ax^2':
        g_bar_x_values = a_average * new_x_values \
                           * new_x_values
    elif g_function == 'ax^2 + b':
        g_bar_x_values = a_average * new_x_values \
                         * new_x_values + b_average
    else:
        raise ValueError("Invalid g_function: " + g_function)
    
    f_x_values = np.sin(np.pi * new_x_values)
    bias = np.mean((g_bar_x_values - f_x_values)
                   * (g_bar_x_values - f_x_values))
    print("bias =", bias)
    
    average_g_D_minus_g_bar_values = np.zeros(trials, dtype=float)
    for i in range(trials):
        if g_function == 'b':
            g_D_x_values = b_values[i]
        elif g_function == 'ax':
            g_D_x_values = a_values[i] * new_x_values
        elif g_function == 'ax + b':
            g_D_x_values = a_values[i] * new_x_values \
                             + b_values[i]
        elif g_function == 'ax^2':
            g_D_x_values = a_values[i] * new_x_values \
                              * new_x_values
        elif g_function == 'ax^2 + b':
            g_D_x_values = a_values[i] * new_x_values \
                             * new_x_values + b_values[i]
        else:
            raise ValueError("Invalid g_function: " + g_function)
        average_g_D_minus_g_bar_value = \
            np.mean((g_D_x_values - g_bar_x_values)
                    * (g_D_x_values - g_bar_x_values))
        average_g_D_minus_g_bar_values[i] = \
            average_g_D_minus_g_bar_value
    variance = np.mean(average_g_D_minus_g_bar_values)
    print("variance =", variance)
    
    print("E_out =", bias + variance)

For Problem 7

In [ ]:
bias_and_variance_analysis_v2('b')

In [ ]:
bias_and_variance_analysis_v2('ax')

In [ ]:
bias_and_variance_analysis_v2('ax + b')

In [ ]:
bias_and_variance_analysis_v2('ax^2')

In [ ]:
bias_and_variance_analysis_v2('ax^2 + b')